<a href="https://colab.research.google.com/github/marcus-wuertz/Analytics-Code/blob/main/DS4220_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $\color{darkblue}{\text{Question 1}}$

## $\color{orange}{\text{a)}}$

#### $\color{darkblue}{\text{Naive Bayes}}$

In [ ]:
bc_data <- read.csv('/content/sample_data/breast cancer.csv')

In [ ]:
#install.packages('e1071')
library(e1071)

# convert target variable to be a factor (catergorical) instead of character
bc_data$diagnosis <- as.factor(bc_data$diagnosis)

# Naive Bayes model
nb_model <- naiveBayes(diagnosis ~ ., data = bc_data, type='class')
nb_pred <- predict(nb_model, newdata=bc_data, type = 'class')

In [ ]:
# Naive Bayes prediction table
table(nb_pred, bc_data$diagnosis)

       
nb_pred   B   M
      B 343  22
      M  14 190

#### $\color{darkblue}{\text{Linear Discriminant Analysis}}$

In [ ]:
#install.packages('MASS')
library(MASS)

# LDA model
lda_model <- lda(diagnosis ~ ., data = bc_data)
lda_pred <- predict(lda_model, newdata = bc_data)


In [ ]:
# Getting class labels for LDA model
lda_probs <- as.data.frame(lda_pred$posterior)

# Update class variable based on conditions
lda_probs$class <- ifelse(lda_probs$B > 0.5, "M", "B")

# LDA prediction table
table(lda_pred$class, bc_data$diagnosis)

   
      B   M
  B 355  18
  M   2 194

#### $\color{darkblue}{\text{Logistic Regression}}$

In [ ]:
# Logistic Regression Model
lr_model <- glm(diagnosis ~ ., data = bc_data, family = 'binomial')


Warning message:
“glm.fit: algorithm did not converge”
Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”


In [ ]:
# Predict logistic regression
lr_pred <- predict(lr_model, type = 'response')

# Get class labels for LR model
lr_class <- ifelse(lr_pred > 0.5, 'M', 'B')

# Get prediction summary // not a probability, gives us the summary in odds
summary(lr_pred)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.0000  0.0000  0.0000  0.3726  1.0000  1.0000 

In [ ]:
# Probabilities
lr_probs <- 1/(1 + exp(-lr_pred))

# Summary // converted to probabilities
summary(lr_probs)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5000  0.5000  0.5000  0.5861  0.7311  0.7311 

In [ ]:
print(lr_model)


Call:  glm(formula = diagnosis ~ ., family = "binomial", data = bc_data)

Coefficients:
            (Intercept)                       id              radius_mean  
              9.535e+01               -8.422e-07               -1.856e+03  
           texture_mean           perimeter_mean                area_mean  
              2.613e+01               -5.121e+00                1.850e+01  
        smoothness_mean         compactness_mean           concavity_mean  
              2.267e+04               -1.781e+04                7.035e+03  
    concave.points_mean            symmetry_mean   fractal_dimension_mean  
              1.449e+04               -7.373e+03                1.617e+04  
              radius_se               texture_se             perimeter_se  
             -7.733e+02               -2.418e+02               -3.304e+02  
                area_se            smoothness_se           compactness_se  
              5.952e+01                2.557e+03                3.204e+04  

# $\color{darkblue}{\text{Question 2}}$

In [ ]:
#install.packages('caret')
library(caret)

# Naive Bayes model probabilities
nb_mod_prob <- naiveBayes(diagnosis ~ ., data = bc_data, type='raw')
nb_prob <- as.data.frame(predict(nb_mod_prob, newdata=bc_data, type = 'raw'))
summary(nb_prob)

       B                M         
 Min.   :0.0000   Min.   :0.0000  
 1st Qu.:0.0000   1st Qu.:0.0000  
 Median :1.0000   Median :0.0000  
 Mean   :0.6396   Mean   :0.3604  
 3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   :1.0000   Max.   :1.0000  

In [ ]:
# LDA probabilities
summary(lda_probs$B)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
0.000000 0.009607 0.994359 0.656730 0.999795 1.000000 

In [ ]:
# LR probabilites
summary(lr_probs)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.5000  0.5000  0.5000  0.5861  0.7311  0.7311 

The average probability of a tumor being benign is 63.96% for the Naive Bayes model, 65.67% for the LDA model, and 58.61% for the LR model. The three classification models all agree that there is difference betwen the groups, with the chance of the diagnosis being benign being greater than that of it being malignant.

## $\color{orange}{\text{b)}}$

In [ ]:
#install.packages("FSA")
#install.packages("nortest")
library("FSA")
library("nortest")

# AD test for tumors that are Benign(B) across each tumor (id).
benign_test <- bc_data[bc_data$diagnosis == "B", ]
ad.test(benign_test$id)

# AD test for tumors that are Malignant(M) across each tumor.
malignant_test <- bc_data[bc_data$diagnosis == "M", ]
ad.test(malignant_test$id)


	Anderson-Darling normality test

data:  benign_test$id
A = 102.32, p-value < 2.2e-16



	Anderson-Darling normality test

data:  malignant_test$id
A = 57.024, p-value < 2.2e-16


Since the p-values are less than 0.05, we reject the null hypothesis that the distribution is normal. Will perform a non-parametric test to determine if the means are different.

In [ ]:
# Conducting KW test
kruskal.test(diagnosis ~ id, data = bc_data)


	Kruskal-Wallis rank sum test

data:  diagnosis by id
Kruskal-Wallis chi-squared = 568, df = 568, p-value = 0.4921


Because the KW test has a p-value greater than 0.05, we do not have evidence to reject the null hypothesis that the means are equal.

# $\color{darkblue}{\text{Question 3}}$

In [ ]:
# lr model coefficients
#print(lr_model)

# Interpret numeric variable 'AreaWorst'
AreaWorst <-  -6.344e+00
print(AreaWorst)

# Getting the odds for AreaWorst
print(exp(AreaWorst))


[1] -6.344
[1] 0.001757259


Thus, the odds of Malignancy increase by a factor of 1.001757259 for addition to the area worst mean.

In [ ]:
# Interpret numeric variable 'RadiusSe'
RadiusSe <- -7.733e+02
print(RadiusSe)

# Getting the odds for RadiusSe
print(exp(RadiusSe))

[1] -773.3
[1] 0


Addition to the radius se mean does not increase the odds of malignancy.